#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces
  

In [94]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from config import pwd, user
from datetime import date
import openpyxl
import requests
from bs4 import BeautifulSoup
import json

In [95]:
#Baixa os anexos XML dos emails

save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [100]:
# Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        produto ['Codigo Produto'] = det.find("./ns:prod/ns:cProd", ns).text
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produtos.append(produto)
    return produtos
        


# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns)
            pasta_destino = os.path.join("notas/")

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")



fornecedores_pesados = [
    'IND E COM DE TUBOS E CONEXOES FORT.COM',
    'VOTORANTIM CIMENTOS SA',
    'CABOQUINHO MATERIAIS PARA CONSTRUCAO'
]


produtos = pd.DataFrame(todos_produtos)
produtos = produtos[~produtos['Fornecedor'].isin(fornecedores_pesados)]

produtos = produtos.drop_duplicates(subset='Codigo Produto', keep='first')

In [ ]:

for index, produto in produtos:
    fornecedor = produto['Fornecedor']
    codigo_produto = produto['Codigo Produto']
    descricao = produto['Descrição']

if fornecedor == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':
   url = f'https://www.construdigi.com.br/produto/{codigo_produto}/{descricao}',
elif fornecedor == 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO':
    url = f'https://msbitaqua.com.br/produto/{codigo_produto}/{descricao}'
elif fornecedor == "CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU":
    url = f'https://www.construja.com.br/produto/{codigo_produto}/{descricao}',




     Codigo Produto                               Descrição Valor_unitário  \
0            355135          BROCA ACO RAPIDO DEWALT  2,0MM          2.451   
1            355143          BROCA ACO RAPIDO DEWALT  2,5MM          1.958   
2            371262     SUPORTE BRASFORMA P/FORNO BR SBR3.6         23.206   
3            310310  ASSENTO METASUL ALMOFADADO OVAL BRANCO          29.69   
4            290742    COTOVELO ESGOTO FORTLEV B 50 X 90 BR           1.05   
...             ...                                     ...            ...   
9587         111134                PREGO ACO 10X10 C/C KG**  77.9930000000   
9588         163400                  MASSA RAPIDA CZ 1,25KG  37.6650000000   
9590          11189                   PREGO ACO 18X24 C/C %  32.6690000000   
9592          12027           SUP EXT ZINCO C/33 PLATIBANDA   9.1940000000   
9593          11182                   PREGO ACO 12X12 C/C %   8.2700000000   

     Código de Barras                       Sku  \
0       7896

In [ ]:
url = "https://construja.com.br/produto/17011/irwin-broca-ao-rapido-60mm"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Procurando o <script> com type="application/ld+json"
script = soup.find("script", type="application/json")
data = json.loads(script.string) if script else {}

produto = data.get("props", {}).get("pageProps", {}).get("produto", {})


marca = next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "Não disponível")
peso = produto.get("pesoBruto", "Não disponível")
codigo_barras = produto.get("codBarra", "Não disponível")


'IRWIN'